# Install

In [ ]:
!wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
!sudo apt-get install -y libsndfile1-dev ffmpeg fluidsynth
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy
!sudo apt-get update
!sudo apt-get install -y lilypond
!pip install pretty_midi
!pip install music21 verovio

# Import

In [ ]:
import libarchive
import pydot
import cartopy
import os
from google.colab import files
from IPython import display as dsp
from IPython.display import IFrame
from music21 import converter
import verovio
from IPython.display import SVG
import zipfile

# Environment

In [ ]:
!./bin/micromamba create -y -n omni python=3.8 pip

In [ ]:
!./bin/micromamba run -n omni pip install "numpy==1.23.5" cython
!./bin/micromamba run -n omni pip install git+https://github.com/Music-and-Culture-Technology-Lab/omnizart.git

In [ ]:
!./bin/micromamba run -n omni omnizart download-checkpoints

# Upload

In [ ]:
ff = files.upload()
uploaded_audio = list(ff.keys())[0].replace(".mp3", "")

!./bin/micromamba run -n omni bash -c "test -f '{uploaded_audio}.wav' && rm '{uploaded_audio}.wav'"
!./bin/micromamba run -n omni ffmpeg -i "{uploaded_audio}.mp3" "{uploaded_audio}.wav" &>/dev/null

dsp.Audio(uploaded_audio + ".mp3") if os.path.exists(uploaded_audio + ".mp3") else None

# Transcribe

In [ ]:
mode = "music-piano-v2" #@param ["music-piano", "music-piano-v2", "music-assemble", "chord", "drum", "vocal", "vocal-contour", "beat"]

model = ""
if mode.startswith("music"):
  mode_list = mode.split("-")
  mode = mode_list[0]
  model = "-".join(mode_list[1:])

model_path = {
    "piano": "Piano",
    "piano-v2": "PianoV2",
    "assemble": "Stream",
    "pop-song": "Pop",
    "": None
}[model]

if model_path:
  !./bin/micromamba run -n omni omnizart "$mode" transcribe "$uploaded_audio".wav --model-path "$model_path"
else:
  !./bin/micromamba run -n omni omnizart "$mode" transcribe "$uploaded_audio".wav

SF2_FILE = "general_soundfont.sf2"
if not os.path.exists(SF2_FILE):
  print("Downloading soundfont...")
  !./bin/micromamba run -n omni curl "https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General/MuseScore_General.sf2" -o general_soundfont.sf2

synth_name = f"{uploaded_audio}_synth.wav"

python_script_synth = f"""
import os
import wave
import numpy as np
from pretty_midi import PrettyMIDI

uploaded_audio = "{uploaded_audio}"
SF2_FILE = "{SF2_FILE}"
synth_name = uploaded_audio + "_synth.wav" # Fixed: Use string concatenation
mode = "{mode}"

def write_audio_file(filename, sample_rate, data):
    with wave.open(filename, 'w') as wf:
        wf.setnchannels(1)      # Mono
        wf.setsampwidth(2)      # 16-bit
        wf.setframerate(sample_rate)
        if data.dtype != np.int16:
            data = np.int16(data * 32767)
        wf.writeframes(data.tobytes())

if mode == "vocal-contour":
    if os.path.exists(uploaded_audio + "_trans.wav"): # Fixed: Use string concatenation
        os.rename(uploaded_audio + "_trans.wav", synth_name)
    else:
        print(f"Warning: {uploaded_audio}_trans.wav not found for vocal-contour mode. No synthesized audio will be generated.")
else:
    print("Synthesizing MIDI...")
    midi = PrettyMIDI(uploaded_audio + ".mid") # Fixed: Use string concatenation
    raw_wav = midi.fluidsynth(fs=44100, sf2_path=SF2_FILE)
    write_audio_file(synth_name, 44100, raw_wav)
"""

!./bin/micromamba run -n omni python -c "$python_script_synth"

!./bin/micromamba run -n omni ffmpeg -i "$synth_name" "tmp_synth.mp3" &> /dev/null
!mv tmp_synth.mp3 "$uploaded_audio"_synth.mp3

out_name = synth_name.replace(".wav", ".mp3")
print(f"Finished: {out_name}")

dsp.Audio(filename=out_name, rate=44100)

In [ ]:
uploaded = files.upload()
midi_path = list(uploaded.keys())[0]

s = converter.parse(midi_path)

musicxml_path = s.write('musicxml')

with open(musicxml_path, 'r', encoding='utf-8') as f:
    mxl_data = f.read()

toolkit = verovio.toolkit()
toolkit.loadData(mxl_data)

svg = toolkit.renderToSVG(1)
SVG(svg)

In [ ]:
page_count = toolkit.getPageCount()
print("Count：", page_count)

os.makedirs("score_svg", exist_ok=True)

for i in range(1, page_count + 1):
    svg = toolkit.renderToSVG(i)
    filename = f"score_svg/score_page_{i}.svg"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(svg)

zip_name = "score_svg_all.zip"
with zipfile.ZipFile(zip_name, "w") as zf:
    for fn in os.listdir("score_svg"):
        zf.write(os.path.join("score_svg", fn), arcname=fn)

files.download(zip_name)